In [1]:
from  selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options

import time
import operator
import pandas as pd

import re


In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
chrome_path = r"D:\saxou\Documents\Programlar\PYTHON HACK\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(chrome_path, options = options)

#chrome_path = r"D:\saxou\Documents\Programlar\PYTHON HACK\chromedriver_win32\chromedriver.exe"
#driver = webdriver.Chrome(chrome_path)

In [3]:
driver.get("https://www.propertyinvestorapp.co.uk/login")

In [4]:
username = "kaanozturk676@gmail.com"
password = "iSw9Lqs48YvU9s5"

In [5]:
wait = WebDriverWait(driver, 10)

In [6]:
#login
print("logging in")

username_form = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[1]/div/form/div[1]/div/div/input")
password_form = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[1]/div/form/div[2]/div/div/input")

wait.until(EC.element_to_be_clickable((By.ID, 'submit-login')))

username_form.send_keys(username)
password_form.send_keys(password)

login_bttn = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[1]/div/form/div[4]/div/button")
login_bttn.click()
print("done")

In [7]:
#get to the first page then get the links of non-sold property adverts for all pages
print("scraping links of the adverts")

driver.get("https://www.propertyinvestorapp.co.uk/properties/availableproperties")
page_bttns = driver.find_elements_by_class_name("page-item")

max_page_num = int(page_bttns[-2].find_element_by_css_selector("a").text)
#print(max_page_num)

property_links = []

i = 1
while i < max_page_num:
    driver.implicitly_wait(10)
    page_bttns = driver.find_elements_by_class_name("page-item")

    next_page_bttn = page_bttns[-1].find_element_by_css_selector("a").get_attribute("href")
    #print(next_page_bttn)
    #######################
    
    property_cards = driver.find_elements_by_id("customer-property-card")

    for property_card in property_cards:
        card_header = property_card.find_element_by_class_name("card-header").text
        if "Sold" not in card_header:
            #print(card_header)
            property_link = property_card.find_element_by_id("view-property-button").get_attribute("href")
            property_links.append(property_link)

    #######################
    print(i)
    driver.get(next_page_bttn)
    
    i += 1
print("done")

In [8]:
df = pd.DataFrame(columns=["Address/City/Postal Code","Weblink","Number of Rooms","Return on Invesment","Purchase Price","Market Price","Pct Below Market Value","Potential Cash Flow","Finders Fee","Current Gross Income","Potential Gross Income","Cash Flow Required"])

In [9]:
print("scraping data from the adverts")

for i,link in enumerate(property_links[:]):
    now = time.time()

    driver.get(link)
    driver.implicitly_wait(10)

    tmp = {}

    tmp["Weblink"] = property_links[i]

    address_city_postalcode = driver.find_element_by_class_name("mt-4").find_element_by_class_name("label-paragraph").text
    #print(address_city_postalcode)
    tmp["Address/City/Postal Code"] = address_city_postalcode

    infos_on_first_section = driver.find_elements_by_class_name("mb-3")[0].find_elements_by_css_selector("strong")
    for info in infos_on_first_section:
        #print(info.text)
        if "Market Value =" in info.text:
            market_value = info.text.replace("Market","").replace("Price","").replace("=","").replace(" ","").replace("Value","")
            market_value = market_value.lower()
            market_value = re.sub(r'[a-z]+', '', market_value, re.I)
            tmp["Market Price"] = market_value

        if "Purchase Price" in info.text:
            purchase_price = info.text.replace("Purchase","").replace("Price","").replace("=","").replace(" ","")
            purchase_price = purchase_price.lower()
            purchase_price = re.sub(r'[a-z]+', '', purchase_price, re.I)
            tmp["Purchase Price"] = purchase_price

        if "Finders Fee" in info.text or "Finder's Fee" in info.text or "Finder’s Fee" in info.text:
            finders_fee = info.text.replace("Finders","").replace("Fee","").replace(" ","").replace("=","").replace("Finder’s","").replace("Finder's","")
            finders_fee = finders_fee.lower()
            finders_fee = re.sub(r'[a-z]+', '', finders_fee, re.I)
            tmp["Finders Fee"] = finders_fee

        if "Below Market Value" in info.text and "%" in info.text:
            pct_below_market_value = info.text.replace("Below","").replace("Market","").replace("Value","").replace("Opportunity","").replace(" ","")
            pct_below_market_value = pct_below_market_value.lower()
            pct_below_market_value = re.sub(r'[a-z]+', '', pct_below_market_value, re.I)
            tmp["Pct Below Market Value"] = pct_below_market_value

        if "Current Gross Income" in info.text:
            current_gross_income = info.text.replace("Current","").replace("Gross","").replace("Income","").replace("=","").replace(" ","")
            current_gross_income = current_gross_income.lower()
            current_gross_income = re.sub(r'[a-z]+', '', current_gross_income, re.I)
            tmp["Current Gross Income"] = current_gross_income

        if "Potential Gross Income" in info.text:
            potential_gross_income = info.text.replace("Potential","").replace("Gross","").replace("Income","").replace("=","").replace(" ","")
            potential_gross_income = potential_gross_income.lower()
            potential_gross_income = re.sub(r'[a-z]+', '', potential_gross_income, re.I)
            tmp["Potential Gross Income"] = potential_gross_income

    infos_on_card = driver.find_element_by_class_name("property-investment-info-div").find_element_by_class_name("row").find_elements_by_class_name("prop-capital-fields")
    for info in infos_on_card:
        if "Return on Investment" in info.find_element_by_class_name("label-paragraph").text:
            return_on_invesment = info.find_element_by_css_selector("h3").text.replace("Return","").replace("Investment","").replace("on","").replace("=","").replace(" ","")
            return_on_invesment = return_on_invesment.lower()
            return_on_invesment = re.sub(r'[a-z]+', '', return_on_invesment, re.I)
            tmp["Return on Invesment"] = return_on_invesment

        if "Potential Cashflow" in info.find_element_by_class_name("label-paragraph").text:
            potential_cashflow = info.find_element_by_css_selector("h3").text.replace("Potential","").replace("Flow","").replace("Cash","").replace("=","").replace(" ","")
            potential_cashflow = potential_cashflow.lower()
            potential_cashflow = re.sub(r'[a-z]+', '', potential_cashflow, re.I)
            tmp["Potential Cash Flow"] = potential_cashflow

    for mb_3 in driver.find_elements_by_class_name("mb-3"):
        label = mb_3.find_element_by_class_name("property-info-label")
        if "capital required to get involved" in label.text.lower():
            cash_flow_required = mb_3.find_elements_by_class_name("ql-align-center")[0].text
            new = ""
            for letter in cash_flow_required:
                if not(letter.isalpha()):
                    new+=letter
            cash_flow_required = new.replace("=","").replace(" ","").replace("()","")
            tmp["Cash Flow Required"] = cash_flow_required

        if "rental" in label.text.lower():
            num_of_rooms = mb_3.find_element_by_class_name("text-field-content").text
            num_of_rooms = num_of_rooms.lower()
            num_of_rooms = re.sub(r'[a-z]+', '', num_of_rooms, re.I)
            tmp["Number of Rooms"] = num_of_rooms

    df = df.append(tmp,ignore_index=True)
    #print(i+1)

    later = time.time()
    difference = int(later - now)
    #print(difference)

    print("advert_num: " + str(i+1) + " / " +  "time spent on scraping the advert: " + str(difference))

print("done")

advert_num: 1 / time spent on scraping the advert: 15
advert_num: 2 / time spent on scraping the advert: 4
advert_num: 3 / time spent on scraping the advert: 14
advert_num: 4 / time spent on scraping the advert: 3
advert_num: 5 / time spent on scraping the advert: 2
advert_num: 6 / time spent on scraping the advert: 5
advert_num: 7 / time spent on scraping the advert: 4
advert_num: 8 / time spent on scraping the advert: 6
advert_num: 9 / time spent on scraping the advert: 10
advert_num: 10 / time spent on scraping the advert: 8
advert_num: 11 / time spent on scraping the advert: 14
advert_num: 12 / time spent on scraping the advert: 2
advert_num: 13 / time spent on scraping the advert: 4
advert_num: 14 / time spent on scraping the advert: 5
advert_num: 15 / time spent on scraping the advert: 5
advert_num: 16 / time spent on scraping the advert: 59
advert_num: 17 / time spent on scraping the advert: 4
advert_num: 18 / time spent on scraping the advert: 6
advert_num: 19 / time spent on s

In [10]:
driver.quit()

In [11]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,Address/City/Postal Code,Weblink,Number of Rooms,Return on Invesment,Purchase Price,Market Price,Pct Below Market Value,Potential Cash Flow,Finders Fee,Current Gross Income,Potential Gross Income,Cash Flow Required
0,"Albany Road, Doncaster, DN4 0QW",https://www.propertyinvestorapp.co.uk/properti...,was previously rented out as follows:\nroom ...,29%,"£149,000","£167,000",11%,"£1,142","£5,000","£1,840£22,080",NaN,"£47,720"
1,"Bevan Street, Port Talbot, SA12 6ND",https://www.propertyinvestorapp.co.uk/properti...,"= £450 pcm = £5,400 per annum\nas a hmo = £3...",17%,"£68,000",NaN,NaN,£280,"£4,000","£5,400","£11,819","£28,040"
2,"Offerton Street, Sunderland, SR4 7EA",https://www.propertyinvestorapp.co.uk/properti...,are currently let out as follows\nofferton s...,23%,NaN,"£195,000",18%,"£1,050","£6,000",NaN,"£17,400","£53,800"
3,"Leylands Lane, Bradford, BD9 5PX",https://www.propertyinvestorapp.co.uk/properti...,", 7 the rooms have long term professional ten...",130%,NaN,NaN,NaN,NaN,"£3,000",NaN,NaN,"£7,800"
4,"Trimsaran, Llanelli, SA17 4DW",https://www.propertyinvestorapp.co.uk/properti...,NaN,NaN,"£180,000",NaN,NaN,NaN,"£6,000",NaN,NaN,"£102,400"
5,"Regent Road, Blackpool, FY1 4NB",https://www.propertyinvestorapp.co.uk/properti...,5 are currently let out\nflat 1 – rented at ...,22%,"£215,000","£250,000",14%,"£1,194","£4,000",NaN,"£20,784","£66,200"
6,"Picton Road, Liverpool, L15 4LH",https://www.propertyinvestorapp.co.uk/properti...,"are all vacant, doubles and could be rented ...",128%,NaN,NaN,NaN,NaN,"£4,000",NaN,NaN,"£6,350"
7,"Raikes Parade, Blackpool, FY1 4EY",https://www.propertyinvestorapp.co.uk/properti...,7 are currently let out \nflat 1 – rented at...,27%,"£320,000","£350,000",NaN,"£2,172","£5,000","£35,664",NaN,"£96,600"
8,"Princess Street, Luton, Bedfordshire, LU1 5AT",https://www.propertyinvestorapp.co.uk/properti...,8 are currently let out as follows :\napartm...,13%,"£835,000","£950,000",12%,"£3,197","£7,500","£63,264",NaN,"£292,500"
9,"Winterhey Avenue, Wallasey, CH44 4DX",https://www.propertyinvestorapp.co.uk/properti...,is rented as follows :\nroom 1 – rented at £...,35%,"£110,000","£135,000",19%,"£1,111","£4,000","£16,380","£20,540","£37,800"


In [12]:
df.to_csv('propertyinvestorapp.csv',encoding='utf-8-sig')
print("The file is updated.")

In [15]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d"))

Current date and time : 
2020-08-03
